In this notedbook we will be doing some sentiment analysis in python using two different techniques:

1. VADER - Bag of words approach
2. Roberta Pretrained Model from Huggingface

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
# nltk.download('punkt')
# # nlrk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import re

In [31]:
# Data loading
df = pd.read_csv('reviews.csv')
df = df.head(500)
print(df.shape)

(500, 6)


In [32]:
# basic NLTK
example = df['comments'][60]
print(example)

Needs a lot of maintenance. 4 different keys required to get in and out, kind of excessive. Water pressure is poor. Not very clean. Parking is driving up in a circle 8 times… kind of annoying


In [41]:
nltk.word_tokenize(example,preserve_line=True)

['Needs',
 'a',
 'lot',
 'of',
 'maintenance.',
 '4',
 'different',
 'keys',
 'required',
 'to',
 'get',
 'in',
 'and',
 'out',
 ',',
 'kind',
 'of',
 'excessive.',
 'Water',
 'pressure',
 'is',
 'poor.',
 'Not',
 'very',
 'clean.',
 'Parking',
 'is',
 'driving',
 'up',
 'in',
 'a',
 'circle',
 '8',
 'times…',
 'kind',
 'of',
 'annoying']

In [ ]:
print(df.shape)

In [ ]:
# Data Cleaning
def clean_text(text):
  if isinstance(text, str):
    text = re.sub(r'[^a-zA-Z\s]', '', text) # remove punctuations
    text = text.lower() # lowercase
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return text
  return ''

df['cleaned_comments'] = df['comments'].apply(clean_text)

In [ ]:
# Data Preparation
df = df[df['cleaned_comments'] != ''] # removing null reviews
X = df['cleaned_comments']
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
# Text Proprocessing
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
  tokens = word_tokenize(text)
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
  return ' '.join(filtered_tokens)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [ ]:
# Model Training
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

In [ ]:
# Test Set Evaluation
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

# VADER Model

# RoBERTa Model

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

d:\[02] Projects\[02] Data Science\[03] Machine Learning\machine-learning\review-sentiment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
